# Sorting Example using Ray

Ensure Ray is installed.

This example demonstrates a two stage sorting technique.  The first stage tasks opens a file to sort the numbers and the second stage task takes the single-file sorted lists and merges them into one list.

In [1]:
import ray

In [9]:
def sort_integers_in_file(file_path):
    import time
    time.sleep(2)
    with open(file_path, "r") as f:
        # first sort integers in file as they are read
        sorted_integers = []
        n = 0
        for line in f:
            integer = int(line)
            index_to_insert = n
            # TODO: since the integers are sorted, could use binary search to optimize the following code
            for i, v in enumerate(sorted_integers):
                if integer < v:
                    index_to_insert = i
                    break
            n = n + 1
            sorted_integers.insert(index_to_insert, integer)
    return sorted_integers

In [5]:
def second_stage_sort(sorted_lists):
    sorted_lists = [ray.get(s) for s in sorted_lists]
    print(f"First stage sort results: {sorted_lists}")
    sorted_lists = list(sorted_lists)
    final_sort = []

    while True:
        min_index = 0
        min_value = sorted_lists[min_index][0]

        for i, sorted_list in enumerate(sorted_lists[1:]):
            if sorted_list[0] < min_value:
                min_index = i + 1
                min_value = sorted_list[0]
        final_sort.append(min_value)
        del sorted_lists[min_index][0]

        if len(sorted_lists[min_index]) == 0:
            del sorted_lists[min_index]
            if len(sorted_lists) == 0:
                break
    return final_sort

In [10]:
data_files_to_sort = ["data/numbers/1.txt", "data/numbers/2.txt", "data/numbers/3.txt", "data/numbers/4.txt"]

sort_tasks = []
for file_to_sort in data_files_to_sort:
    task = ray.remote(sort_integers_in_file).remote(file_to_sort)
    # print(ray.get(task))
    sort_tasks.append(task)

final_sort_task = ray.remote(second_stage_sort).remote(sort_tasks)

[2, 5, 11, 44, 333]
[1, 44, 222, 3333, 55555]
[3, 4, 22, 11111, 5555555]
[2222, 33333, 55555, 444444, 1111111]


(second_stage_sort pid=67385) First stage sort results: [[2, 5, 11, 44, 333], [1, 44, 222, 3333, 55555], [3, 4, 22, 11111, 5555555], [2222, 33333, 55555, 444444, 1111111]]


In [7]:
ray.get(final_sort_task)

[1,
 2,
 3,
 4,
 5,
 11,
 22,
 44,
 44,
 222,
 333,
 2222,
 3333,
 11111,
 33333,
 55555,
 55555,
 444444,
 1111111,
 5555555]